In [1]:
import config
import json
import requests
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool, GenerateContentResponse
from typing import Dict, List

c:\Users\domin\anaconda3\envs\gemini\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load JSON Table Data
with open('all_info.json') as json_file:
    dataset = json.load(json_file)

# Configure Gemini
genai.configure(api_key=config.api_key)

In [3]:
# FunctionDeclarations
get_column_titles = FunctionDeclaration(
    name="get_column_titles",
    description="Fetches all column titles from the specified table.",
    parameters={
        "type": "object",
        "properties": {
            "table_id": {
                "type": "string",
                "description": "The ID of the table to extract column titles from.",
            }
        },
        "required": ["table_id"],
    },
)

get_column_descriptions = FunctionDeclaration(
    name="get_column_descriptions",
    description="Fetches column titles and their descriptions from the specified table.",
    parameters={
        "type": "object",
        "properties": {
            "table_id": {
                "type": "string",
                "description": "The ID of the table to extract column descriptions from.",
            }
        },
        "required": ["table_id"],
    },
)

get_table_summary = FunctionDeclaration(
    name="get_table_summary",
    description="Fetches the summary of the specified table.",
    parameters={
        "type": "object",
        "properties": {
            "table_id": {
                "type": "string",
                "description": "The ID of the table to extract the summary from.",
            }
        },
        "required": ["table_id"],
    },
)

get_specific_column_description = FunctionDeclaration(
    name="get_specific_column_description",
    description="Fetches the description of a specific column from the specified table.",
    parameters={
        "type": "object",
        "properties": {
            "table_id": {
                "type": "string",
                "description": "The ID of the table to fetch the column description from.",
            },
            "column_name": {
                "type": "string",
                "description": "The name of the column to fetch the description of.",
            },
        },
        "required": ["table_id", "column_name"],
    },
)

In [4]:
# Function Definitions
def fetch_table_by_id(table_id: str):
    """Utility function to fetch a table by its ID."""
    for table in dataset:
        if table.get("table_id") == table_id:
            return table
    raise ValueError(f"Table with ID '{table_id}' not found.")

def get_column_titles(table_id: str):
    table = fetch_table_by_id(table_id)
    return [
        col["TitleColumn"]
        for col in table.get("columns", [])
        if "TitleColumn" in col
    ]

def get_column_descriptions(table_id: str):
    table = fetch_table_by_id(table_id)
    return [
        {"title": col["TitleColumn"], "description": col.get("Description")}
        for col in table.get("columns", [])
    ]

def get_table_summary(table_id: str):
    table = fetch_table_by_id(table_id)
    return table.get("summary", "No summary available.")

def get_specific_column_description(table_id: str, column_name: str):
    table = fetch_table_by_id(table_id)
    for col in table.get("columns", []):
        if col.get("TitleColumn", "").lower() == column_name.lower():
            return col.get("Description", "No description available.")
    return f"Column '{column_name}' not found in table '{table_id}'."


In [12]:
def execute_function(function_call):
    function_name = function_call.name
    args = function_call.args
    
    if function_name == "get_column_titles":
        return get_column_titles(**args)
    elif function_name == "get_column_descriptions":
        return get_column_descriptions(**args)
    elif function_name == "get_table_summary":
        return get_table_summary(**args)
    elif function_name == "get_specific_column_description":
        return get_specific_column_description(**args)
    else:
        raise ValueError(f"Unknown function name: {function_name}")

In [6]:
function_calling_tools = Tool(
    function_declarations=[
        get_column_titles,
        get_column_descriptions,
        get_table_summary,
        get_specific_column_description,
      ],
)

model = genai.GenerativeModel("gemini-1.5-flash", tools=function_calling_tools)

In [15]:
def process_query_with_function_calls(chat, question):
    response = chat.send_message(question)
    
    if response.candidates[0].content.parts:
        for part in response.candidates[0].content.parts:
             if "function_call" in part:  # Check if the part includes a function_call
                function_call = part.function_call

                # Execute and process the function call
                function_result = execute_function(function_call)
                result_text = json.dumps(function_result, indent=2)
                follow_up = f"Use the results from the function call ({function_call.name}) to make further decisions. The results are as follows:\n{result_text}"
                
                # Send the result back to the model
                print(function_call.name)
                response = chat.send_message(follow_up)
    
    return response.candidates[0].content.parts

In [19]:
chat = model.start_chat()
qs = "What are the columns in table ID 37738ENG. Furthermore what do these columns entail?"
output = process_query_with_function_calls(chat, qs)
print(output)

get_column_titles
get_column_descriptions


[text: "Based on the descriptions:\n\n* **Vegetables:** This column lacks a description.  More information is needed to understand what it represents.\n\n* **Periodes:** This column also lacks a description.  Further clarification is required.\n\n* **Gross yield:** This column represents the yield of vegetables, measured in million kg.\n\n* **Cropping area:** This column represents the total cropping area, which is calculated differently depending on the type of vegetable (e.g., it\'s counted multiple times for repeatedly sown areas but only once for strawberries or mushrooms).\n\nTo get a complete understanding of the table, we need to obtain descriptions for the \"Vegetables\" and \"Periodes\" columns.  The description for \"Cropping area\" is quite detailed, but may need further clarification if specific scenarios are unclear.\n"
]